In [ ]:
%pip install semantic-link-labs --q

In [ ]:
import sempy_labs as labs
import sempy.fabric as fabric

In [ ]:
from sempy_labs.tom import connect_semantic_model

def copy_relationships(
    source_dataset: str,
    target_dataset: str,
    source_workspace: str = None,
    target_workspace: str = None
):
    """
    Copy all relationships from the source semantic model to the target semantic model.
    """
    with connect_semantic_model(dataset=source_dataset, workspace=source_workspace) as src_tom:
        for rel in src_tom.model.Relationships:
            print(f"Copying relationship: {rel.Name}")
            src_tom.copy_object(
                object=rel,
                target_dataset=target_dataset,
                target_workspace=target_workspace
            )
    print("All relationships have been copied.")


def copy_calculated_columns(
    source_dataset: str,
    target_dataset: str,
    source_workspace: str = None,
    target_workspace: str = None
):
    """
    Copy all calculated columns from the source semantic model to the target semantic model.
    Uses sempy.fabric.list_columns to reliably identify calculated columns.
    """
    calc_cols_df = fabric.list_columns(source_dataset, workspace=source_workspace)[['Table Name', 'Column Name', 'Type']]
    calculated_columns = set(
        tuple(x) for x in calc_cols_df.query('Type == "Calculated"')[['Table Name', 'Column Name']].values
    )

    with connect_semantic_model(dataset=source_dataset, workspace=source_workspace) as src_tom:
        for table in src_tom.model.Tables:
            for column in table.Columns:
                if (table.Name, column.Name) in calculated_columns:
                    print(f"Copying calculated column: {column.Name} from table: {table.Name}")
                    src_tom.copy_object(
                        object=column,
                        target_dataset=target_dataset,
                        target_workspace=target_workspace
                    )
    print("All calculated columns have been copied.")

def copy_calculated_tables(
    source_dataset: str,
    target_dataset: str,
    source_workspace: str = None,
    target_workspace: str = None
):
    """
    Copy all calculated tables from the source semantic model to the target semantic model.

    """

    calculated_tables = set(
        fabric.list_tables(source_dataset, workspace=source_workspace)
              .query('Type == "Calculated Table"')['Name']
    )

    with connect_semantic_model(dataset=source_dataset, workspace=source_workspace) as src_tom:
        for table in src_tom.model.Tables:
            if table.Name in calculated_tables:
                print(f"Copying calculated table: {table.Name}")
                src_tom.copy_object(
                    object=table,
                    target_dataset=target_dataset,
                    target_workspace=target_workspace
                )
    print("All calculated tables have been copied.")

from sempy_labs.tom import connect_semantic_model

def copy_all_measures(
    source_dataset: str,
    target_dataset: str,
    source_workspace: str = None,
    target_workspace: str = None
):
    """
    Copy all measures from every table in the source semantic model to the target semantic model.

    """
    with connect_semantic_model(dataset=source_dataset, workspace=source_workspace) as src_tom:
        for table in src_tom.model.Tables:
            for measure in table.Measures:
                print(f"Copying measure '{measure.Name}' from table '{table.Name}'...")
                try:
                    src_tom.copy_object(
                        object=measure,
                        target_dataset=target_dataset,
                        target_workspace=target_workspace)
                except:
                    print(f"Error with {measure.Name}, check again")
                    continue
                
    print("All measures have been copied.")


In [ ]:
copy_relationships(
source_dataset="model_1",
target_dataset="model_2",
source_workspace="a79cbb27-3bff-4e2c-a0cc-d64bf25ca405",
target_workspace="a79cbb27-3bff-4e2c-a0cc-d64bf25ca405"
)

In [ ]:
copy_calculated_tables(
source_dataset="model_1",
target_dataset="model_2",
source_workspace="a79cbb27-3bff-4e2c-a0cc-d64bf25ca405",
target_workspace="a79cbb27-3bff-4e2c-a0cc-d64bf25ca405"
)

In [ ]:
copy_calculated_columns(
source_dataset="model_1",
target_dataset="model_2",
source_workspace="a79cbb27-3bff-4e2c-a0cc-d64bf25ca405",
target_workspace="a79cbb27-3bff-4e2c-a0cc-d64bf25ca405"
)

In [ ]:
copy_all_measures(
source_dataset="model_1",
target_dataset="model_2",
source_workspace="a79cbb27-3bff-4e2c-a0cc-d64bf25ca405",
target_workspace="a79cbb27-3bff-4e2c-a0cc-d64bf25ca405"
)

In [ ]:
fabric.refresh_tom_cache()